<a href="https://colab.research.google.com/github/thofes/judging_SJ_BA/blob/main/Mark_11_yolo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
#Connect GDrive
from google.colab import drive
drive.mount('/content.gdrive')

#init Repository - YOLOv5
%cd /content


!git clone https://github.com/thofes/yolov5  # clone
%cd yolov5
%pip install -qr requirements.txt  # install

Drive already mounted at /content.gdrive; to attempt to forcibly remount, call drive.mount("/content.gdrive", force_remount=True).
/content
fatal: destination path 'yolov5' already exists and is not an empty directory.
/content/yolov5


Exception falls gar nichts entdeckt wir dim BIld

In [48]:
import os
from os.path import exists, join, basename, splitext
import math
from collections import OrderedDict
import json
import numpy as np
import cv2


PATH = "/content.gdrive/My Drive/Try"
label= []
try:
    label = next(os.walk(PATH))[1]
except StopIteration:
    pass
#data["labels"].append(label)
print(label)

for l in label:
  Path_jumpers = PATH + '/' + l
  print(Path_jumpers)
  Jumpers = []
  try:
    Jumpers = next(os.walk(Path_jumpers))[1]
  except StopIteration:
    pass
  print(Jumpers)

  for j in Jumpers:
    Path_Pic = Path_jumpers + '/' + j
    print(Path_Pic)

    %cd /content/yolov5
    !python detect.py --save-crop --source '{Path_Pic}' --project '{Path_Pic}' --save-txt


    with open(Path_Pic + '/exp/predictions.json', "r") as fp:
        data = json.load(fp)

    # convert lists to numpy arrays
    x_center = np.array(data["x_center"])
    y_center = np.array(data["y_center"])
    x1 = np.array(data["x1"])
    x2 = np.array(data["x2"])
    y1 = np.array(data["y1"])
    y2 = np.array(data["y2"])
    det = np.array(data["det"])
    width = 1920
    hight = 1080




    #detect movement mit check vor und zurück


    athlet_det_index = []
    y_centers = []

    for y in range(21):
      print("loop" , y)
      if det[y] != 0: #wird eventuellenötigt, falls keine detections gemacht wurden
        print("detvorhanden")
        distance_next = []
        distance_index_next = []
        distance_previous = []
        distance_index_previous = []
        valid_dist_index_next = []
        valid_dist_index_previous = []
        valid_dist_index = []
        y_center_per_det = []
        y_center_abs_per_det = []

    # calculate distances and save relevant information

        if y == 0: #beim ersten Bild wird der Abstand der BB zum Folgebild berrechnet

            for x in range(det[y+1]):
                distance_index_previous.append(x) # wird aufgefüllt damit beim Vergleich nur die Indexe der distance_next Liste vorhanden bleiben            
                for i in range(det[y]):
                    distance_next.append(math.sqrt((x_center[sum(det[:y+1])+x]-x_center[sum(det[:y])+i])**2 + (y_center[sum(det[:y+1])+x]-y_center[sum(det[:y])+i])**2))
                    distance_index_next.append(i)
            

        elif y > 0 and y < 20: #bei den mittleren Bildern wird der Abstand der BB zum Vorbild und Folgebild berrechnet
            
            for x in range(det[y+1]):
                
                for i in range(det[y]):
                    distance_next.append(math.sqrt((x_center[sum(det[:y+1])+x]-x_center[sum(det[:y])+i])**2 + (y_center[sum(det[:y+1])+x]-y_center[sum(det[:y])+i])**2))
                    distance_index_next.append(i)

            for x in range(det[y-1]):
                
                for i in range(det[y]):
                    distance_previous.append(math.sqrt((x_center[sum(det[:y-1])+x]-x_center[sum(det[:y])+i])**2 + (y_center[sum(det[:y-1])+x]-y_center[sum(det[:y])+i])**2))
                    distance_index_previous.append(i)
            #print(distance)
            
            
        else: # Beim letzten Bild wird der Abstand zur BB des vorletzten Bildes berechnet
            for x in range(det[y-1]):
                
                for i in range(det[y]):
                    distance_previous.append(math.sqrt((x_center[sum(det[:y-1])+x]-x_center[sum(det[:y])+i])**2 + (y_center[sum(det[:y-1])+x]-y_center[sum(det[:y])+i])**2))
                    distance_index_previous.append(i)



        for x in range(det[y]): # y Koordinaten des Zentrums in Liste speichern
            y_center_per_det.append(y_center[sum(det[:y])+x])
            

        y_centers.append(y_center_per_det) # Erstellung einer Liste mit den Listen der einzelnen y Koordinaten der Detections in jedem Bild

        print("distance_next: ",distance_next)
        print("distance_previous:",distance_previous)
    # Movement detector

        if y == 0: # beim ersten Bild nur vergleich mit dem Nächsten und Einschränkung durch eine Vorgabe der Distanz
            print("first BB")
            for i, x in enumerate(distance_next): # Einschränkung der Detections anhand des Abstandes zum nächsten Bild    
                if x > 20 and x < 75:
                    valid_dist_index.append(distance_index_next[i])


        elif athlet_det_index[y-1] == 999: # falls beim letzten Bild keine Detection gemacht wurde 
            print("last det 999 BP")
            last_det_index = 999
            counter = len(athlet_det_index)
            for index in athlet_det_index: # letzte gültige detection finden
                if index != 999:
                    last_det_index = index
                    counter = counter - 1
                    missings = len(athlet_det_index)-counter
            if last_det_index != 999: # wenn eine vorhanden ist dann zusätzlich die Prüfung zum y Abstand durchführen (Prüfung durch Vorgabe)
                for i, x in enumerate(distance_next):    
                    if x > 20 and x < 75+missings*25 and abs(y_center[sum(det[:y])+i%det[y]]-y_center[sum(det[:y-1-counter])+last_det_index]) < 100:
                        valid_dist_index.append(distance_index_next[i])

            else: # wenn keine vorhanden ist nur eine Einschränkung zur Distanz der Detections im nächsten Bild
            #eventuell besser nur y Koordinate?
                for i, x in enumerate(distance_next):     
                    if x > 20 and x < 75:
                        valid_dist_index.append(distance_index_next[i])

        elif y > 0 and y < 20: # bei den mittleren Bildern werden beide Richtungen eingeschränkt  und zusätzlich wird auch geprüft, ob die Y Koordinaten nicht zu weit abweichen
            print("middel BB")
            for i, x in enumerate(distance_next): # Einschränkung der Detections anhand des Abstandes zum nächsten Bild
                if x > 20 and x < 75 and abs(y_center[sum(det[:y])+i%det[y]]-y_center[sum(det[:y-1])+athlet_det_index[y-1]]) < 100:
                    valid_dist_index_next.append(distance_index_next[i])     

            for i, x in enumerate(distance_previous): # Einschränkung der Detections anhand des Abstandes zum vorherigen Bild
                if x > 20 and x < 75 and abs(y_center[sum(det[:y])+i%det[y]]-y_center[sum(det[:y-1])+athlet_det_index[y-1]]) < 100:
                    valid_dist_index_previous.append(distance_index_previous[i])


            #wird einfach aufsumiert und dann später geordnet
            valid_dist_index = valid_dist_index_next + valid_dist_index_previous
            

        elif y == 20: # beim letzten Bid wieder nur vergleich mit dem vorherigen Bild
            print("last BB")
            for i, x in enumerate(distance_previous): # Einschränkung der Detections anhand des Abstandes zum vorherigen Bild sowie Validierung der Y Koordinate
                if x > 20 and x < 75 and abs(y_center[sum(det[:y])+i%det[y]]-y_center[sum(det[:y-1])+athlet_det_index[y-1]]) < 100:
                    valid_dist_index.append(distance_index_previous[i])




    # in der Liste valid_dist_index sind die Indexe der validen Detections im Bild gespeichert (Validierung mit Movement detector)
        print("validdetList: ", valid_dist_index)

    # Anpassungen der Liste der validen Detections
        if len(valid_dist_index) > 1: # Falls für eine Bildbox mehrere mögliche Verbindungen zum Folgebild gefunden wurden, werden die Duplikate entfernt
            valid_dist_index = list(OrderedDict.fromkeys(valid_dist_index))

        if len(valid_dist_index) == 1: # wenn nur eine vorhanden ist wird diese verwendet
            athlet_det_index.append(valid_dist_index[0])

        if y == 0: # Für das erste Bild
            
            if len(valid_dist_index) > 1: # falls mehrere möglich sind, wird die BB mit der zentralste Y Koordinate gewählt
                for dist in valid_dist_index:
                    y_center_abs_per_det.append(abs(y_center[sum(det[:y])+dist]-hight/2))
                closest = min(y_center_abs_per_det)
                min_index = y_center_abs_per_det.index(closest)            
                athlet_det_index.append(valid_dist_index[min_index]) # index der Detection vom ersten Bild gespeichert
            
            elif len(valid_dist_index) == 0: # falls keine validen aus dem Movement detector vorhanden sind wird jene detections verwendet, wo die Y-Koordinate am zentralsten ist
            #eventuell bessr keine det? 999 scheint so - noch einmal testen
                athlet_det_index.append(999)
                """
                for x in range(det[y]):
                    y_center_abs_per_det.append(abs(y_center[sum(det[:y])+x]-hight/2)) 
                closest = min(y_center_abs_per_det)
                min_index = y_center_abs_per_det.index(closest)
                athlet_det_index.append(min_index) # index der Detection vom ersten Bild gespeichert
                """

        elif y < 21: # Für alle weiteren Bilder 

            if len(valid_dist_index) > 1: # falls mehrere möglich sind, wird jene BB mit dem geringsten Abstand zur Y-Koordinate der vorher geählten BB ausgewählt
                dist_to_previous_y_center = []
                for index in athlet_det_index: # letzte gültige detection finden
                    if index != 999:
                        last_det_index = index            

                for i, x in enumerate(valid_dist_index):
                    dist_to_previous_y_center.append(abs(y_center_per_det[x]-y_center[sum(det[:y-1])+last_det_index]))
                closest = min(dist_to_previous_y_center)
                min_index = dist_to_previous_y_center.index(closest)
                athlet_det_index.append(valid_dist_index[min_index]) # index der Detection wird gespeichert           

                #optional einfach die zentralste Detection laut Y-Kooridnate
                #for x in range(det[y]):
                    #y_center_abs_per_det.append(abs(y_center[sum(det[:y])+x]-hight/2)) 
                #closest = min(y_center_abs_per_det)
                #min_index = y_center_abs_per_det.index(closest)
                #athlet_det_index.append(valid_dist_index[min_index]) # index der Detection vom Bild wird gespeichert

            if len(valid_dist_index) == 0: #falls keine Movements aus den BB erkannt werden - auch zetralste Y-Koordinate oder vergleichen mit dem vorherigen Bild
                athlet_det_index.append(999)



      else:
          athlet_det_index.append(999) # falls keine detections im Bild vorhanden sind
      print("TOFILTER: ", athlet_det_index)










    #crop images
    for i,x in enumerate(athlet_det_index):
        if x != 999: # zuschneiden laut Koordinaten der BB (50 Pixel vergrößert)
            x_1 = x1[sum(det[:i])+x]-50
            x_2 = x2[sum(det[:i])+x]+50
            y_1 = y1[sum(det[:i])+x]-50
            y_2 = y2[sum(det[:i])+x]+50

            if x_1 < 0:
                x_1 = 0

            if x_2 > width:
                x_2 = width

            if y_1 < 0:
                y_1 = 0

            if y_2 > hight:
                y_2 = hight

        else: # falls keine Detection gefunden wurde wird einfach ein kleines Bild ausgeschnitten, dass später wieder 21 Bilder vorhanden sind
            x_1 = 0
            x_2 = 1
            y_1 = 0
            y_2 = 1


        path = Path_Pic + '/' + str(i+1) + '.jpg'
        print(path)
        im = cv2.imread(path)
        #print(x_1, x_2, y_1, y_2)
        crop = im[int(y_1):int(y_2), int(x_1):int(x_2)] 
        #save_dir.parent.mkdir(parents=True, exist_ok=True)  # make directory
        save_path = Path_Pic + '/exp/crop' + str(i+1) + '.jpg'
        cv2.imwrite(save_path, crop) # speicher nur den zugeschnittenen Springer






Die letzten 5000 Zeilen der Streamingausgabe wurden abgeschnitten.
/content.gdrive/My Drive/Try/NoTelemark/1941/1.jpg
6
1
3
/content.gdrive/My Drive/Try/NoTelemark/1941/2.jpg
9
2
2
/content.gdrive/My Drive/Try/NoTelemark/1941/3.jpg
13
3
3
/content.gdrive/My Drive/Try/NoTelemark/1941/4.jpg
16
4
2
/content.gdrive/My Drive/Try/NoTelemark/1941/5.jpg
20
5
3
/content.gdrive/My Drive/Try/NoTelemark/1941/6.jpg
24
6
3
/content.gdrive/My Drive/Try/NoTelemark/1941/7.jpg
27
7
2
/content.gdrive/My Drive/Try/NoTelemark/1941/8.jpg
32
8
4
/content.gdrive/My Drive/Try/NoTelemark/1941/9.jpg
37
9
4
/content.gdrive/My Drive/Try/NoTelemark/1941/10.jpg
42
10
4
/content.gdrive/My Drive/Try/NoTelemark/1941/11.jpg
45
11
2
/content.gdrive/My Drive/Try/NoTelemark/1941/12.jpg
50
12
3
/content.gdrive/My Drive/Try/NoTelemark/1941/13.jpg
55
13
3
/content.gdrive/My Drive/Try/NoTelemark/1941/14.jpg
60
14
3
/content.gdrive/My Drive/Try/NoTelemark/1941/15.jpg
65
15
4
/content.gdrive/My Drive/Try/NoTelemark/1941/16.jpg
6

In [69]:
#without loop

import os
from os.path import exists, join, basename, splitext
import math
from collections import OrderedDict
import json
import numpy as np
import cv2


Path_Pic = "/content.gdrive/My Drive/Try/NoTelemark/554"


#%cd /content/yolov5
#!python detect.py --save-crop --source '{Path_Pic}' --project '{Path_Pic}' --save-txt


with open(Path_Pic + '/exp/predictions.json', "r") as fp:
    data = json.load(fp)

# convert lists to numpy arrays
x_center = np.array(data["x_center"])
y_center = np.array(data["y_center"])
x1 = np.array(data["x1"])
x2 = np.array(data["x2"])
y1 = np.array(data["y1"])
y2 = np.array(data["y2"])
det = np.array(data["det"])
width = 1920
hight = 1080






#detect movement mit check vor und zurück


athlet_det_index = []
y_centers = []

for y in range(21):
  print("loop" , y)
  if det[y] != 0: #wird eventuellenötigt, falls keine detections gemacht wurden
    print("detvorhanden")
    distance_next = []
    distance_index_next = []
    distance_previous = []
    distance_index_previous = []
    valid_dist_index_next = []
    valid_dist_index_previous = []
    valid_dist_index = []
    y_center_per_det = []
    y_center_abs_per_det = []

# calculate distances and save relevant information

    if y == 0: #beim ersten Bild wird der Abstand der BB zum Folgebild berrechnet

        for x in range(det[y+1]):
            distance_index_previous.append(x) # wird aufgefüllt damit beim Vergleich nur die Indexe der distance_next Liste vorhanden bleiben            
            for i in range(det[y]):
                distance_next.append(math.sqrt((x_center[sum(det[:y+1])+x]-x_center[sum(det[:y])+i])**2 + (y_center[sum(det[:y+1])+x]-y_center[sum(det[:y])+i])**2))
                distance_index_next.append(i)
        

    elif y > 0 and y < 20: #bei den mittleren Bildern wird der Abstand der BB zum Vorbild und Folgebild berrechnet
        
        for x in range(det[y+1]):
            
            for i in range(det[y]):
                distance_next.append(math.sqrt((x_center[sum(det[:y+1])+x]-x_center[sum(det[:y])+i])**2 + (y_center[sum(det[:y+1])+x]-y_center[sum(det[:y])+i])**2))
                distance_index_next.append(i)

        for x in range(det[y-1]):
            
            for i in range(det[y]):
                distance_previous.append(math.sqrt((x_center[sum(det[:y-1])+x]-x_center[sum(det[:y])+i])**2 + (y_center[sum(det[:y-1])+x]-y_center[sum(det[:y])+i])**2))
                distance_index_previous.append(i)
        #print(distance)
        
         
    else: # Beim letzten Bild wird der Abstand zur BB des vorletzten Bildes berechnet
        for x in range(det[y-1]):
            
            for i in range(det[y]):
                distance_previous.append(math.sqrt((x_center[sum(det[:y-1])+x]-x_center[sum(det[:y])+i])**2 + (y_center[sum(det[:y-1])+x]-y_center[sum(det[:y])+i])**2))
                distance_index_previous.append(i)



    for x in range(det[y]): # y Koordinaten des Zentrums in Liste speichern
        y_center_per_det.append(y_center[sum(det[:y])+x])
        

    y_centers.append(y_center_per_det) # Erstellung einer Liste mit den Listen der einzelnen y Koordinaten der Detections in jedem Bild

    print("distance_next: ",distance_next)
    print("distance_previous:",distance_previous)
# Movement detector

    if y == 0: # beim ersten Bild nur vergleich mit dem Nächsten und Einschränkung durch eine Vorgabe der Distanz
        print("first BB")
        for i, x in enumerate(distance_next): # Einschränkung der Detections anhand des Abstandes zum nächsten Bild    
            if x > 20 and x < 75:
                valid_dist_index.append(distance_index_next[i])


    elif athlet_det_index[y-1] == 999: # falls beim letzten Bild keine Detection gemacht wurde 
        print("last det 999 BP")
        last_det_index = 999
        counter = len(athlet_det_index)
        for index in athlet_det_index: # letzte gültige detection finden
            if index != 999:
                last_det_index = index
                counter = counter - 1
                missings = len(athlet_det_index)-counter
        if last_det_index != 999: # wenn eine vorhanden ist dann zusätzlich die Prüfung zum y Abstand durchführen (Prüfung durch Vorgabe)
            for i, x in enumerate(distance_next):    
                if x > 20 and x < 75+missings*25 and abs(y_center[sum(det[:y])+i%det[y]]-y_center[sum(det[:y-1-counter])+last_det_index]) < 100:
                    valid_dist_index.append(distance_index_next[i])

        else: # wenn keine vorhanden ist nur eine Einschränkung zur Distanz der Detections im nächsten Bild
        #eventuell besser nur y Koordinate?
            for i, x in enumerate(distance_next):     
                if x > 20 and x < 75:
                    valid_dist_index.append(distance_index_next[i])

    elif y > 0 and y < 20: # bei den mittleren Bildern werden beide Richtungen eingeschränkt  und zusätzlich wird auch geprüft, ob die Y Koordinaten nicht zu weit abweichen
        print("middel BB")
        for i, x in enumerate(distance_next): # Einschränkung der Detections anhand des Abstandes zum nächsten Bild
            if x > 20 and x < 75 and abs(y_center[sum(det[:y])+i%det[y]]-y_center[sum(det[:y-1])+athlet_det_index[y-1]]) < 100:
                valid_dist_index_next.append(distance_index_next[i])     

        for i, x in enumerate(distance_previous): # Einschränkung der Detections anhand des Abstandes zum vorherigen Bild
            if x > 20 and x < 75 and abs(y_center[sum(det[:y])+i%det[y]]-y_center[sum(det[:y-1])+athlet_det_index[y-1]]) < 100:
                valid_dist_index_previous.append(distance_index_previous[i])


        #wird einfach aufsumiert und dann später geordnet
        valid_dist_index = valid_dist_index_next + valid_dist_index_previous
        

    elif y == 20: # beim letzten Bid wieder nur vergleich mit dem vorherigen Bild
        print("last BB")
        for i, x in enumerate(distance_previous): # Einschränkung der Detections anhand des Abstandes zum vorherigen Bild sowie Validierung der Y Koordinate
            if x > 20 and x < 75 and abs(y_center[sum(det[:y])+i%det[y]]-y_center[sum(det[:y-1])+athlet_det_index[y-1]]) < 100:
                valid_dist_index.append(distance_index_previous[i])




# in der Liste valid_dist_index sind die Indexe der validen Detections im Bild gespeichert (Validierung mit Movement detector)
    print("validdetList: ", valid_dist_index)

# Anpassungen der Liste der validen Detections
    if len(valid_dist_index) > 1: # Falls für eine Bildbox mehrere mögliche Verbindungen zum Folgebild gefunden wurden, werden die Duplikate entfernt
        valid_dist_index = list(OrderedDict.fromkeys(valid_dist_index))

    if len(valid_dist_index) == 1: # wenn nur eine vorhanden ist wird diese verwendet
        athlet_det_index.append(valid_dist_index[0])

    if y == 0: # Für das erste Bild
        
        if len(valid_dist_index) > 1: # falls mehrere möglich sind, wird die BB mit der zentralste Y Koordinate gewählt
            for dist in valid_dist_index:
                y_center_abs_per_det.append(abs(y_center[sum(det[:y])+dist]-hight/2))
            closest = min(y_center_abs_per_det)
            min_index = y_center_abs_per_det.index(closest)            
            athlet_det_index.append(valid_dist_index[min_index]) # index der Detection vom ersten Bild gespeichert
        
        elif len(valid_dist_index) == 0: # falls keine validen aus dem Movement detector vorhanden sind wird jene detections verwendet, wo die Y-Koordinate am zentralsten ist
        #eventuell bessr keine det? 999 scheint so - noch einmal testen
            athlet_det_index.append(999)
            """
            for x in range(det[y]):
                y_center_abs_per_det.append(abs(y_center[sum(det[:y])+x]-hight/2)) 
            closest = min(y_center_abs_per_det)
            min_index = y_center_abs_per_det.index(closest)
            athlet_det_index.append(min_index) # index der Detection vom ersten Bild gespeichert
            """

    elif y < 21: # Für alle weiteren Bilder 

        if len(valid_dist_index) > 1: # falls mehrere möglich sind, wird jene BB mit dem geringsten Abstand zur Y-Koordinate der vorher geählten BB ausgewählt
            dist_to_previous_y_center = []
            for index in athlet_det_index: # letzte gültige detection finden
                if index != 999:
                    last_det_index = index            

            for i, x in enumerate(valid_dist_index):
                dist_to_previous_y_center.append(abs(y_center_per_det[x]-y_center[sum(det[:y-1])+last_det_index]))
            closest = min(dist_to_previous_y_center)
            min_index = dist_to_previous_y_center.index(closest)
            athlet_det_index.append(valid_dist_index[min_index]) # index der Detection wird gespeichert           

            #optional einfach die zentralste Detection laut Y-Kooridnate
            #for x in range(det[y]):
                #y_center_abs_per_det.append(abs(y_center[sum(det[:y])+x]-hight/2)) 
            #closest = min(y_center_abs_per_det)
            #min_index = y_center_abs_per_det.index(closest)
            #athlet_det_index.append(valid_dist_index[min_index]) # index der Detection vom Bild wird gespeichert

        if len(valid_dist_index) == 0: #falls keine Movements aus den BB erkannt werden - auch zetralste Y-Koordinate oder vergleichen mit dem vorherigen Bild
            athlet_det_index.append(999)



  else:
      athlet_det_index.append(999) # falls keine detections im Bild vorhanden sind
  print("TOFILTER: ", athlet_det_index)










#crop images
for i,x in enumerate(athlet_det_index):
    if x != 999: # zuschneiden laut Koordinaten der BB (50 Pixel vergrößert)
        x_1 = x1[sum(det[:i])+x]-50
        x_2 = x2[sum(det[:i])+x]+50
        y_1 = y1[sum(det[:i])+x]-50
        y_2 = y2[sum(det[:i])+x]+50

        if x_1 < 0:
            x_1 = 0

        if x_2 > width:
            x_2 = width

        if y_1 < 0:
            y_1 = 0

        if y_2 > hight:
            y_2 = hight

    else: # falls keine Detection gefunden wurde wird einfach ein kleines Bild ausgeschnitten, dass später wieder 21 Bilder vorhanden sind
        x_1 = 0
        x_2 = 1
        y_1 = 0
        y_2 = 1


    path = Path_Pic + '/' + str(i+1) + '.jpg'
    print(path)
    im = cv2.imread(path)
    #print(x_1, x_2, y_1, y_2)
    crop = im[int(y_1):int(y_2), int(x_1):int(x_2)] 
    #save_dir.parent.mkdir(parents=True, exist_ok=True)  # make directory
    save_path = Path_Pic + '/exp/crop' + str(i+1) + '.jpg'
    cv2.imwrite(save_path, crop) # speicher nur den zugeschnittenen Springer

loop 0
detvorhanden
distance_next:  [847.2558641904734, 8.514705643352103, 207.60839276208176, 709.1567373992295, 201.3361660732266, 4.743414228420183, 1209.1125567052381, 1697.9985912915167, 1680.3237644424562]
distance_previous: []
first BB
validdetList:  []
TOFILTER:  [999]
loop 1
detvorhanden
distance_next:  [1640.4145214790915, 1605.4404716054696, 234.10523897566068, 1.581139584109208, 202.72210645653502, 1691.3504047444371, 201.7919738859616, 3.905116534784285, 1686.114231985995, 1627.9760724357836, 1619.2585959034823, 63.78277996497802]
distance_previous: [847.2558641904734, 709.1567373992295, 1209.1125567052381, 8.514705643352103, 201.3361660732266, 1697.9985912915167, 207.60839276208176, 4.743414228420183, 1680.3237644424562]
last det 999 BP
validdetList:  [2]
TOFILTER:  [999, 2]
loop 2
detvorhanden
distance_next:  [1059.1134036472238, 847.3082289934554, 712.6803067454938, 1148.2135709877264, 1635.4757320751041, 7.648533198063732, 204.26701632183497, 1622.6216891826339, 211.09